In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
import re

In [13]:
from tqdm.auto import tqdm

# Load Some Data to preprocess Vietnamese languages

In [2]:
# EMOJI
with open('../DATA/files/emojicon.txt', 'r', encoding="utf8") as file:
    emoji_lst = file.read().split('\n')
    emoji_dict = {}
    for line in emoji_lst:
        key, value = line.split('\t')
        emoji_dict[key] = str(value)

# TEEN CODE
with open('../DATA/files/teencode.txt', 'r', encoding="utf8") as file:
    teen_lst = file.read().split('\n')
    teen_dict = {}
    for line in teen_lst:
        key, value = line.split('\t')
        teen_dict[key] = str(value)

# ENG VIET
with open('../DATA/files/english-vnmese.txt', 'r', encoding="utf8") as file:
    eng_lst = file.read().split('\n')
    eng_dict = {}
    for line in eng_lst:
        key, value = line.split('\t')
        eng_dict[key] = str(value)

# WRONG WORD
with open('../DATA/files/wrong-word.txt', 'r', encoding="utf8") as file:
    wrong_lst = file.read().split('\n')

# STOP WORD
with open('../DATA/files/vietnamese-stopwords.txt', 'r', encoding="utf8") as file:
    stop_lst = file.read().split('\n')

# Load file data:

In [51]:
use_cols = ['product_id','sub_category','price','rating','clean_desc','product_name','clean_prd_name']
clean_prd_df = pd.read_csv('../DATA/clean_products.csv',usecols=use_cols)

In [52]:
clean_prd_df.head()

,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,86250.0,4.9,áo lỗ chiều đường_phố nhiệt_đới tal fit xuất_x...,áo lỗ thun gân form body
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,26800.0,4.9,áo lỗ xuất_xứ việt_nam tổ_chức trách_nhiệm sản...,áo lỗ nam trắng chất_cotton siêu_mát đẹp
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,39500.0,4.8,áo lỗ thương_hiệu tyasuo chiều áo không_thể đư...,áo lỗ nam tyasuo chất vải co_dãn mát không_xù mềm
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,16500.0,4.8,áo lỗ chất_liệu hàng gửi hà_nội áo lỗ nam mặc ...,áo lỗ hàng việt_nam
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ,45000.0,4.8,áo lỗ chiều áo không_thể hàng gửi hà_nội thông...,áo thun nam thể_thao lỗ mẫu mới đẹp


## Check na values from clean data

In [53]:
drop_idx = clean_prd_df[clean_prd_df['clean_prd_name'].apply(lambda x: isinstance(x,float))].index

In [54]:
clean_prd_df = clean_prd_df.drop(index=drop_idx)

In [55]:
clean_prd_df = clean_prd_df.reset_index(drop=True)

# Cosine_similarity
- link https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
- link https://en.wikipedia.org/wiki/Cosine_similarity

In [7]:
tf_model = TfidfVectorizer(analyzer='word', stop_words=stop_lst)

In [8]:
tfidf_matrix = tf_model.fit_transform(raw_documents=clean_prd_df['clean_prd_name'])

C:\ProgramData\Anaconda3\envs\env39\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['danh', 'mục', 'nam', 'shopee', 'thời', 'trang'] not in stop_words.
  warnings.warn(


In [9]:
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [11]:
cosine_similarities[0].shape

(49594,)

In [12]:
cosine_similarities.shape

(49594, 49594)

In [21]:
# với mỗi sản phẩm, lấy 3 sản phẩm tương quan nhất
results = {}

for idx, row in tqdm(clean_prd_df.iterrows(),'Calcualte the high correaltion',total=clean_prd_df.shape[0]):
    similar_indices = cosine_similarities[idx].argsort()[-6:-1]
    similar_items = [(cosine_similarities[idx][i]) for i in similar_indices]
    similar_items = [(cosine_similarities[idx][i], clean_prd_df.index[i]) for i in similar_indices]
    #print(similar_items[0:])
    results[idx] = similar_items[0:]

Calcualte the high correaltion:   0%|          | 0/49594 [00:00<?, ?it/s]

In [22]:
results[0]

[(0.5371489791665923, 898),
 (0.5454050100363411, 1068),
 (0.5527234870850822, 401),
 (0.5604763876794088, 7329),
 (0.5623337185323803, 268)]

## Save file

In [23]:
# Save the array to a file
np.save('../DATA/cosine_sim_prd.npy', results)

## Apply Cosine Similarity

In [24]:
def item(id_prd,df):
    
    return text

In [56]:
final_df = clean_prd_df.reset_index().rename(columns={'index':'item_id'})
final_df.head()

,item_id,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
0,0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,86250.0,4.9,áo lỗ chiều đường_phố nhiệt_đới tal fit xuất_x...,áo lỗ thun gân form body
1,1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,26800.0,4.9,áo lỗ xuất_xứ việt_nam tổ_chức trách_nhiệm sản...,áo lỗ nam trắng chất_cotton siêu_mát đẹp
2,2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,39500.0,4.8,áo lỗ thương_hiệu tyasuo chiều áo không_thể đư...,áo lỗ nam tyasuo chất vải co_dãn mát không_xù mềm
3,3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,16500.0,4.8,áo lỗ chất_liệu hàng gửi hà_nội áo lỗ nam mặc ...,áo lỗ hàng việt_nam
4,4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ,45000.0,4.8,áo lỗ chiều áo không_thể hàng gửi hà_nội thông...,áo thun nam thể_thao lỗ mẫu mới đẹp


In [68]:
final_df.loc[final_df['item_id'] == 1,'product_name'].values[0]

'Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp'

In [69]:
def recommend_cosine(prd_id,
                     df,
                     cosine_array, num = 5):
    text =  df.loc[df['item_id'] == prd_id,'product_name'].values[0]
    print(f'Recommending {num} products similar to ***{text}***')
    recs = cosine_array[prd_id][:num]
    for rec in recs:
        #print(rec[1])
        print(f"Recommnended:\tItem ID: {rec[1]}, {df.loc[df['item_id'] == rec[1],'product_name'].values[0]}, (score: {rec[0]})\n")

In [71]:
recommend_cosine(prd_id=2001,df=final_df,cosine_array=results,num=4)

Recommending 4 products similar to ***áo sọc nam***
Recommnended:	Item ID: 474, áo 3 lỗ nam sọc ngang Mã: NB073, (score: 0.7646320602976147)

Recommnended:	Item ID: 41193, Áo Sơ Mi Nam Dài Tay Kẻ Sọc Thời Trang, (score: 0.8196531776925334)

Recommnended:	Item ID: 35176, Quần thể thao kẻ sọc nam 1 sọc - 2 sọc - 3 sọc, (score: 0.8400904703451335)

Recommnended:	Item ID: 18173, Áo Thun Nam Kẻ Sọc Thời Trang Cy33, (score: 0.8549251132460183)

